In [5]:
import numpy as np

In [26]:
def EL_HO(R,a):
    return a + R*R*(0.5-2*a*a) #Local energy for harmonic oscillator

def updateWalker_HO(R,d,a,N):
    dis = np.random.normal(0,d,N) #proposed displacement
    p = np.exp(-2*a*((R+dis)**2-R**2)) #acceptence probability for harmonic oscillator
    r = np.random.rand(N)
    R += (r<p)*dis

def updateVarParams_HO(R,EL,a,gamma):
    dEda = 2 * (np.mean(-R*R*EL)-EL.mean()*np.mean(-R*R))
    return a - gamma * dEda, dEda

def updateVarParams_HO_2(R2EL,EL,R2,a,gamma):
    dEda = 2 * (-R2EL+EL*R2)
    return a - gamma * dEda, dEda
    

## Harmonic Oscillator code with saving all walker locations

In [33]:
N = 400 #Number of walkers
T = 30000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

a = 0.6 #Variational parameter harmonic oscillator
d = a/2 #mean step length
gamma = 0.5 #damped steepest descent parameter
epsilon = 1e-12 # absolute value of derivative tolerance
itmax = 12;

dEda = 1; it = 0;
while((abs(dEda) >= epsilon) & (it<itmax)):
    print('a = ',a)
    Ri = np.random.uniform(-4*a,4*a,N)
    R = np.empty(T*N)
    EL = np.empty(T*N)
    for t in range(0,T0):
        updateWalker_HO(Ri,d,a,N)
        
    for t in range(0,T): #loop over number of displacement attempts per walker
        updateWalker_HO(Ri,d,a,N)
        R[t*N:(t+1)*N] = Ri
        
    EL = EL_HO(R,a)
    a,dEda = updateVarParams_HO(R,EL,a,gamma)
    it+=1
    


a =  0.6
a =  0.523245430313
a =  0.501693524249
a =  0.500004461361
a =  0.499999998779
a =  0.499999999991
a =  0.5


In [34]:
print('mean EL: ',EL.mean())
print('var  EL: ',EL.var())

mean EL:  0.5
var  EL:  1.58950291413e-26


## Harmonic Oscillator without saving all walker locations (in between averaging)
This is in fact slower than above for N=400, T=30000, and about equally fast with N=4000, T=30000.
No point in doing this further.

In [29]:
N = 400 #Number of walkers
T = 30000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

a = 0.6 #Variational parameter harmonic oscillator
d = a/2 #mean step length
gamma = 0.5 #damped steepest descent parameter
epsilon = 1e-12 # absolute value of derivative tolerance
itmax = 12;

dEda = 1; it = 0;
while((abs(dEda) >= epsilon) & (it<itmax)):
    print('a = ',a)
    R = np.random.uniform(-4*a,4*a,N) #Initialize walker positions
    for t in range(0,T0): # first T0 steps to equilibriate
        updateWalker_HO(R,d,a,N)
        
    EL = 0
    VarEL = 0
    R2EL = 0
    R2 = 0
    for t in range(0,T): #loop over number of displacement attempts per walker
        updateWalker_HO(R,d,a,N)
        ELit = EL_HO(R,a)
        EL += np.mean(ELit)
        VarEL += np.var(ELit)
        R2EL += np.mean(R*R*ELit)
        R2 += np.mean(R*R)
        
    EL /= T
    VarEL /= T
    R2EL /= T
    R2 /= T
    
    a,dEda = updateVarParams_HO_2(R2EL,EL,R2,a,gamma)
    it+=1

a =  0.6
a =  0.523600193214
a =  0.501470489469
a =  0.500002512537
a =  0.499999985416
a =  0.500000000047
a =  0.5


In [30]:
print('mean EL: ',EL)
print('var  EL: ',VarEL)

mean EL:  0.5
var  EL:  5.60705829874e-27
